# Termowizja

## Prosta analiza obrazu termowizyjnego

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def I__median_and_morfology(I):
    I_median = cv2.medianBlur(I, 3)

    kernel = np.ones((3,3),np.uint8)
    I_result = cv2.erode(I_median,kernel)
    I_result = cv2.dilate(I_median,kernel)

    return I_result

In [3]:
def is_below(main_stat, sub_stat):
    return 0 < sub_stat[1] - main_stat[1] - main_stat[3] < 100

In [4]:
def is_between_edges(main_stat, sub_stat):
    return sub_stat[0] + sub_stat[2] > main_stat[0] and sub_stat[0] < main_stat[0] + main_stat[2]

In [5]:
def update_right_bottom(right_bottom, sub_stat):
    return ((sub_stat[0] + sub_stat[2]) if (right_bottom[0] < sub_stat[0] + sub_stat[2]) else right_bottom[0]), ((sub_stat[1] + sub_stat[3]) if (right_bottom[1] < sub_stat[1] + sub_stat[3]) else right_bottom[1])

def update_left_top(left_top, sub_stat):
    return sub_stat[0] if left_top[0] > sub_stat[0] else left_top[0], left_top[1]

def update_rectangle_tops(left_top, right_bottom, sub_stat):
    return update_left_top(left_top, sub_stat), update_right_bottom(right_bottom, sub_stat)


In [8]:
# patrzymy czy pod dołem wykrytego człowieka nie ma innego wykrytego obiektu
# jeśli jest to klasyfikujemy go jako część wykrytego człowieka i uktaulniamy współrzędne prostokąta

def I_indexation(I_input, I):
    retval, labels, stats, centroids = cv2.connectedComponentsWithStats(I)

    I_VIS = I_input # copy of the input image
    if (stats.shape[0] > 1): # are there any objects
        for (i, stat) in enumerate(stats):

            if i == 0 :
                continue
            if stat[3] / stat[2] > 2:
                left_top, right_bottom = (stat[0], stat[1]), (stat[0] + stat[2], stat[1] + stat[3])
                for (j, st) in enumerate(stats):
                    if (j == 0):
                        continue
                    if (is_below(stat, st) and is_between_edges(stat, st)):
                        left_top, right_bottom = update_rectangle_tops(left_top, right_bottom, st)
                cv2.rectangle(I_VIS,left_top, right_bottom,(255,0,0),2)
    return I_VIS, np.uint8(labels / retval * 255)

In [9]:

cap = cv2.VideoCapture('vid1_IR.avi')
while(cap.isOpened()):
    ret, frame = cap.read()
    if not ret:
        break
    G = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    (T, I) = cv2.threshold(G, 35,255,cv2.THRESH_BINARY)
    I = I__median_and_morfology(I)
    I, tmp = I_indexation(I, I)
    cv2.imshow('IR',I)
    if cv2.waitKey(1) & 0xFF == ord('q'): # break the loop when the ’q32 Termowizja’ key is pressed
        break
cap.release()
cv2.destroyAllWindows()